# Проект спринта 7

- Автор: Колдашев Виталий Андреевич
- Дата: 20.12.2024
- Ссылка на github: [https://github.com/PsychoCoderrr/YPracticumProjects/blob/main/YPracticumProject.ipynb]

### Цели и задачи проекта


- Загрузка и знакомство с данными: Проверить данные на соответствие описанию, наличие пропусков и корректность типов.
- Предобработка данных: Привести названия столбцов к удобному формату, обработать пропуски, ошибки и дубликаты, преобразовать данные к корректным типам
- Фильтрация данных: Отобрать игры, выпущенные в 2000–2013 годах.
- Категоризация данных: Разделить игры по уровням оценок пользователей и критиков, определить топ-7 платформ по количеству выпущенных игр.
- Итоговый вывод: Составить описание среза данных и новых добавленных полей, подготовить итоговую версию очищенного и обработанного датасета для анализа.



### Описание данных

- Name — название игры.
- Platform — платформа, на которой выпущена игра.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах копий).
- EU sales — продажи в Европе (в миллионах копий).
- JP sales — продажи в Японии (в миллионах копий).
- Other sales — продажи в других регионах (в миллионах копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — возрастной рейтинг по классификации ESRB.

### Содержимое проекта

- Загрузка и знакомство с данными
- Предобработка данных
- Фильтрация данных
- Категоризация данных
- Итоговый вывод

## 1. Загрузка данных и знакомство с ними



In [1]:
import pandas as pd

In [2]:
df_start_version = pd.read_csv('https://code.s3.yandex.net//datasets/new_games.csv')

In [3]:
#вывод первых пяти строк датасета
df_start_version.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
df_start_version.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [5]:
row_count_start = df_start_version.shape[0] #будет использоваться в дальнейшем

Для анализа были предоставлены данные примерно на 16900 строк, в столбцах Critic Score, User Score и Rating можно заметить значительное кол-во пропущенных элементов

По моему мнению, названия столбцов в таблице достаточно понятно и правильно отражают суть данных, которые в них содержатся

Но можно заметить, что в столбцах EU sales и JP sales используется неправильный тип данных, должен быть float64, а также в столбце User Score должен быть тип данных float64

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [6]:
df_start_version.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

Названия столбцов корректны, однако их требуется привести к стилю snake case

In [7]:
#переименование столбцов и проверка того, что теперь столбцы называны корректно
columns_snake_case = ['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales'
, 'critic_score', 'user_score', 'rating']
df_start_version.columns = columns_snake_case
df_start_version.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

Общее кол-во строк и кол-во значений в столбцах na_sales, eu_sales, jp_sales и other_sales совпадают, это означает, что в этих столбцах нет пропусков, однако в них могут строковые значения по типы unknown, следовательно будем приводить к типу float64, чтобы оставить пропуски на месте этих значений

In [8]:
#замена типов данных в столбцах eu_sales, jp_sales и na_sales
df_start_version['eu_sales'] = pd.to_numeric(df_start_version['eu_sales'], errors='coerce')
df_start_version['jp_sales'] = pd.to_numeric(df_start_version['jp_sales'], errors='coerce')
df_start_version['na_sales'] = pd.to_numeric(df_start_version['na_sales'], errors='coerce')


Также изменим тип данных в столбце user_score

In [9]:
#замена типов данных в столбце user_score
df_start_version['user_score'] = pd.to_numeric(df_start_version['user_score'], errors='coerce')
#проверка корректной смены данных
df_start_version.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


### 2.3. Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.

In [10]:
missing_absolute = df_start_version.isna().sum()
missing_absolute

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [11]:
missing_relative = df_start_version.isna().sum() / len(df_start_version) * 100
missing_relative

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

Количество пропусков в столбцах name, year_of_realese, genre, eu_sales, jp_sales очень малы, поэтому строки с пропусками в данных столбцах можно просто удалить, это не сильно повлияет на итоговый результат, скорее всего эти пропуски связаны с невнимательностью заполнения, а для каких-то игр неизвестностью данных.

In [12]:
df_start_version = df_start_version.dropna(subset=['name', 'year_of_release', 'genre', 'eu_sales', 'jp_sales'])

Осталось обработать пропуски в столбцах critic_score, user_score и rating. Я бы использовал здесь индикаторы, в столбцах critic_score, user_score я бы использовал индикатор -1, так как рейтинг не может быть отрицательным, а в rating я бы использовал индикатор unknown

In [13]:
df_start_version['critic_score'] = df_start_version['critic_score'].fillna(-1)
df_start_version['user_score'] = df_start_version['user_score'].fillna(-1)
df_start_version['rating'] = df_start_version['rating'].fillna('unknown')
#вывод информации для проверки корректности проведения операций с пропусками
df_start_version.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16669 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16669 non-null  object 
 1   platform         16669 non-null  object 
 2   year_of_release  16669 non-null  float64
 3   genre            16669 non-null  object 
 4   na_sales         16669 non-null  float64
 5   eu_sales         16669 non-null  float64
 6   jp_sales         16669 non-null  float64
 7   other_sales      16669 non-null  float64
 8   critic_score     16669 non-null  float64
 9   user_score       16669 non-null  float64
 10  rating           16669 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


после проведенной работы с пропусками видим, что пропусков не осталось

### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [14]:
unique_name = df_start_version['genre'].unique()
unique_name

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

In [15]:
unique_name = df_start_version['platform'].unique()
unique_name

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [16]:
unique_name = df_start_version['rating'].unique()
unique_name

array(['E', 'unknown', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

После анализа видно, что неявное дублирование есть в столбце genre, следовательно в этом столбце надо привести данные к верхнему регистру

In [17]:
df_start_version['genre'] = df_start_version['genre'].str.upper()

In [18]:
#удаление явных дубликатов из датасета
df_start_version = df_start_version.drop_duplicates()
df_start_version.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16434 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16434 non-null  object 
 1   platform         16434 non-null  object 
 2   year_of_release  16434 non-null  float64
 3   genre            16434 non-null  object 
 4   na_sales         16434 non-null  float64
 5   eu_sales         16434 non-null  float64
 6   jp_sales         16434 non-null  float64
 7   other_sales      16434 non-null  float64
 8   critic_score     16434 non-null  float64
 9   user_score       16434 non-null  float64
 10  rating           16434 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.5+ MB


In [19]:
row_count_final = df_start_version.shape[0]

In [20]:
absolute_cleaned_rows = row_count_start - row_count_final
absolute_cleaned_rows

522

In [21]:
relative_cleaned_rows = (absolute_cleaned_rows / row_count_start) * 100 #получаю ответ в процентах
relative_cleaned_rows

3.0785562632696393

## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [22]:
#провели отбор по году
mask = ((df_start_version['year_of_release'] >= 2000) & (df_start_version['year_of_release'] <= 2013))
df_actual = df_start_version[mask].copy()  # copy здесь добавлен только с целью того, чтобы мы впоследствии работали с копией 
#вывод первых элементов нового датасета для того, чтобы убедиться в корректности проведенной операции
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,SPORTS,41.36,28.96,3.77,8.45,76.0,8.0,E
2,Mario Kart Wii,Wii,2008.0,RACING,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,SPORTS,15.61,10.93,3.28,2.95,80.0,8.0,E
6,New Super Mario Bros.,DS,2006.0,PLATFORM,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,MISC,13.96,9.18,2.93,2.84,58.0,6.6,E


## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [23]:
def categorize_user_score(score):
    if 8 <= score <= 10:
        return 'высокая'
    elif 3 <= score < 8:
        return 'средняя'
    elif 0 <= score < 3:
        return 'низкая'
    else:
        return 'рейтинг неизвестен'

df_actual.loc[:, 'user_score_category'] = df_actual['user_score'].apply(categorize_user_score)
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category
0,Wii Sports,Wii,2006.0,SPORTS,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая
2,Mario Kart Wii,Wii,2008.0,RACING,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая
3,Wii Sports Resort,Wii,2009.0,SPORTS,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая
6,New Super Mario Bros.,DS,2006.0,PLATFORM,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая
7,Wii Play,Wii,2006.0,MISC,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя
...,...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,GC,2003.0,SHOOTER,0.01,0.00,0.00,0.00,-1.0,-1.0,T,рейтинг неизвестен
16949,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,PLATFORM,0.01,0.00,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен
16950,SCORE International Baja 1000: The Official Game,PS2,2008.0,RACING,0.00,0.00,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен
16952,LMA Manager 2007,X360,2006.0,SPORTS,0.00,0.01,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен


In [24]:
#категоризация по оценкам критиков
def categorize_critic_score(score):
    if 80 <= score <= 100:
        return 'высокая'
    elif 30 <= score < 80:
        return 'средняя'
    elif 0 <= score < 30:
        return 'низкая'
    else:
        return 'рейтинг неизвестен'

df_actual.loc[:, 'critic_score_category'] = df_actual['critic_score'].apply(categorize_critic_score)
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
0,Wii Sports,Wii,2006.0,SPORTS,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая,средняя
2,Mario Kart Wii,Wii,2008.0,RACING,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая,высокая
3,Wii Sports Resort,Wii,2009.0,SPORTS,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая,высокая
6,New Super Mario Bros.,DS,2006.0,PLATFORM,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая,высокая
7,Wii Play,Wii,2006.0,MISC,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя,средняя
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16947,Men in Black II: Alien Escape,GC,2003.0,SHOOTER,0.01,0.00,0.00,0.00,-1.0,-1.0,T,рейтинг неизвестен,рейтинг неизвестен
16949,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,PLATFORM,0.01,0.00,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен,рейтинг неизвестен
16950,SCORE International Baja 1000: The Official Game,PS2,2008.0,RACING,0.00,0.00,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен,рейтинг неизвестен
16952,LMA Manager 2007,X360,2006.0,SPORTS,0.00,0.01,0.00,0.00,-1.0,-1.0,unknown,рейтинг неизвестен,рейтинг неизвестен


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [25]:
grouped_by_user_category = df_actual.groupby('user_score_category').agg('count')
grouped_by_user_category

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,critic_score_category
user_score_category,,,,,,,,,,,,
высокая,2282,2282,2282,2282,2282,2282,2282,2282,2282,2282,2282,2282
низкая,116,116,116,116,116,116,116,116,116,116,116,116
рейтинг неизвестен,6296,6296,6296,6296,6296,6296,6296,6296,6296,6296,6296,6296
средняя,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078,4078


In [26]:
grouped_by_critic_category = df_actual.groupby('critic_score_category').agg('count')
grouped_by_critic_category

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category
critic_score_category,,,,,,,,,,,,
высокая,1686,1686,1686,1686,1686,1686,1686,1686,1686,1686,1686,1686
низкая,55,55,55,55,55,55,55,55,55,55,55,55
рейтинг неизвестен,5610,5610,5610,5610,5610,5610,5610,5610,5610,5610,5610,5610
средняя,5421,5421,5421,5421,5421,5421,5421,5421,5421,5421,5421,5421


- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [27]:
grouped_by_platform = df_actual.groupby('platform').agg('count')
sorted_by_count_platform = grouped_by_platform.sort_values('name', ascending=False)
sorted_by_count_platform.head(7)

,name,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
platform,,,,,,,,,,,,
PS2,2126,2126,2126,2126,2126,2126,2126,2126,2126,2126,2126,2126
DS,2117,2117,2117,2117,2117,2117,2117,2117,2117,2117,2117,2117
Wii,1275,1275,1275,1275,1275,1275,1275,1275,1275,1275,1275,1275
PSP,1179,1179,1179,1179,1179,1179,1179,1179,1179,1179,1179,1179
X360,1118,1118,1118,1118,1118,1118,1118,1118,1118,1118,1118,1118
PS3,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087,1087
GBA,810,810,810,810,810,810,810,810,810,810,810,810


## 5. Итоговый вывод

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

В ходе анализа был исследован датасет new_games.csv, в ходе работы была проведена работа над очисткой данных от пропусков и дубликатов, добавлена фильтрация данных по году для того, чтобы отслеживать только актуальные игры.
Также в ходе категоризации были добавлены новые поля user_score_category и critic_score_category, которые отражают категоризацию данных по оценке либо пользователей, либо критиков.
Так же были выделены топ-7 платформ для игр, а именно PS2, DS, Wii, PSP, X360, PS3, GBA.